Mixture of pandas and [sqlite](https://www.sqlite.org/about.html) to examine the df 

limitations of sqlite: I can't do much of the data cleansing in sqlite. I can't do DDL as well (dropping columns and changing datatypes is done in pandas). I'll also include what the sql statement would be to achieve the same results anytime i use pandas. You won't be able to run the SQL cell codes. 2 links. 

https://www.simplilearn.com/tutorials/sql-tutorial/what-is-sqlite#:~:text=Limitations%20of%20SQLite&text=SQLite%20only%20allows%20normal%20file,support%20FOR%20EACH%20STATEMENT%20triggers.

https://www.sqlite.org/limits.html


need to explain in my file description that there are individual value counts in this column

### Importing Neccessary Libraries

In [1]:
import pandas as pd
import sqlite3

### Reading In the Dataset 

In [2]:
df = pd.read_csv('Daily_Inmates_In_Custody.csv')
df

,INMATEID,ADMITTED_DT,DISCHARGED_DT,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SEALED,SRG_FLG,TOP_CHARGE,INFRACTION
0,20197307,03/23/2023 02:49:00 PM,NaN,MAX,Y,B,M,21,DE,N,Y,110-125.25,Y
1,20223604,03/16/2023 09:14:00 PM,NaN,MED,Y,O,M,70,DE,N,N,130.50,N
2,20031187,06/11/2023 12:38:00 AM,NaN,MED,Y,O,M,31,DE,N,Y,220.16,N
3,20223353,03/02/2023 01:57:34 AM,NaN,NaN,Y,O,M,23,DE,N,N,110-125.25,Y
4,20070053,09/05/2021 12:50:57 AM,NaN,MED,N,O,M,32,DE,N,N,110-125.25,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,20140924,07/28/2022 02:08:38 AM,NaN,MED,Y,W,M,44,CS,N,N,270.25,N
6142,20008122,07/13/2023 08:47:04 PM,NaN,MIN,N,O,M,32,DE,N,N,120.00,N
6143,20206710,07/13/2023 07:21:01 PM,NaN,MAX,N,O,M,23,DE,N,N,120.05,N
6144,20220543,09/07/2022 04:43:55 PM,NaN,MAX,N,O,M,20,DE,N,N,NaN,Y


### Establish a connection to the SQLite database file 'crime_database.db'

You can create your own database file by running the code below and it'll run properly. The naming of your *.db* file can be whatever you want

In [3]:
conn = sqlite3.connect('crime_database.db')

### Convert the DF to a SQL Table

In [4]:
df.to_sql('mytable', conn, if_exists='replace', index=False)


6146

The output of the cell above shows the number of rows in the SQL Table. The actual SQL query is below:

In [ ]:
SELECT COUNT(*) FROM mytable

### Examining the Dataframes Info

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   INMATEID            6146 non-null   int64  
 1   ADMITTED_DT         6146 non-null   object 
 2   DISCHARGED_DT       0 non-null      float64
 3   CUSTODY_LEVEL       6113 non-null   object 
 4   BRADH               6146 non-null   object 
 5   RACE                6128 non-null   object 
 6   GENDER              6128 non-null   object 
 7   AGE                 6146 non-null   int64  
 8   INMATE_STATUS_CODE  6146 non-null   object 
 9   SEALED              6146 non-null   object 
 10  SRG_FLG             6146 non-null   object 
 11  TOP_CHARGE          5975 non-null   object 
 12  INFRACTION          6146 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 624.3+ KB


Takeaways:

- The discharged date column has 0 values and should be removed

- The admitted date column needs its data type to be converted to a datetime and then broken out

The SQL alternative for the above code depends on the specific SQL Database you are using. For example, in MySql, this format would query your table data types:

In [ ]:
SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
    table_schema = 'yourDatabaseName' and table_name = 'yourTableName';


To get the non-null counts of each column in SQL, use this query:

In [ ]:
SELECT COUNT('column name') FROM mytable

    WHERE 'column name' IS NOT NULL;

Drop the **Discharged Date** Column 

In [6]:
df = df.drop(columns='DISCHARGED_DT', axis=1)
df.head()

,INMATEID,ADMITTED_DT,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SEALED,SRG_FLG,TOP_CHARGE,INFRACTION
0,20197307,03/23/2023 02:49:00 PM,MAX,Y,B,M,21,DE,N,Y,110-125.25,Y
1,20223604,03/16/2023 09:14:00 PM,MED,Y,O,M,70,DE,N,N,130.50,N
2,20031187,06/11/2023 12:38:00 AM,MED,Y,O,M,31,DE,N,Y,220.16,N
3,20223353,03/02/2023 01:57:34 AM,NaN,Y,O,M,23,DE,N,N,110-125.25,Y
4,20070053,09/05/2021 12:50:57 AM,MED,N,O,M,32,DE,N,N,110-125.25,N


Here's how the above code would be done in SQL:

In [ ]:
ALTER TABLE mytable
    DROP COLUMN DISCHARGED_DT;

Convert the **Admitted Date** to Datetime 

In [7]:
df['ADMITTED_DT'] = pd.to_datetime(df['ADMITTED_DT'])

df.dtypes


INMATEID                       int64
ADMITTED_DT           datetime64[ns]
CUSTODY_LEVEL                 object
BRADH                         object
RACE                          object
GENDER                        object
AGE                            int64
INMATE_STATUS_CODE            object
SEALED                        object
SRG_FLG                       object
TOP_CHARGE                    object
INFRACTION                    object
dtype: object

Converting the **Admitted Date** Column to Datetime in **SQL**

In [ ]:
UPDATE mytable

    SET ADMITTED_DT = CAST(ADMITTED_DT AS DATETIME);

### Splitting our *ADMITTED_DT* into a *Date* and *Time* Column

In [8]:
# Create new columns for date and time
df['Date'] = df['ADMITTED_DT'].dt.date
df['Time'] = df['ADMITTED_DT'].dt.time

# Drop the original column
df = df.drop(columns='ADMITTED_DT')
df.head()


,INMATEID,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SEALED,SRG_FLG,TOP_CHARGE,INFRACTION,Date,Time
0,20197307,MAX,Y,B,M,21,DE,N,Y,110-125.25,Y,2023-03-23,14:49:00
1,20223604,MED,Y,O,M,70,DE,N,N,130.50,N,2023-03-16,21:14:00
2,20031187,MED,Y,O,M,31,DE,N,Y,220.16,N,2023-06-11,00:38:00
3,20223353,NaN,Y,O,M,23,DE,N,N,110-125.25,Y,2023-03-02,01:57:34
4,20070053,MED,N,O,M,32,DE,N,N,110-125.25,N,2021-09-05,00:50:57


In [ ]:
-- Adding the Date column
ALTER TABLE mytable
    ADD COLUMN Date DATE;

-- Adding the Time column 
ALTER TABLE mytable
    ADD COLUMN Time TIME;

-- Update the new columns with date and time values
UPDATE mytable
SET Date = DATE(ADMITTED_DT), -- Extracts the date part from ADMITTED_DT
    Time = TIME(ADMITTED_DT); -- Extracts the time part from ADMITTED_DT

-- Drop the original column 
ALTER TABLE mytable
    DROP COLUMN ADMITTED_DT; 


### Column Level EDA

Let's examine the **Inmate ID** Column to see if any of the inmates was incarcerated more than once

In [9]:
query = "SELECT INMATEID, COUNT(*) as Count FROM mytable GROUP BY INMATEID HAVING COUNT(*) > 1;"
result = pd.read_sql_query(query, conn)

print(result)

Empty DataFrame
Columns: [INMATEID, Count]
Index: []


We can see above the the resulting dataframe is empty. Since there's only a single occurrence of each inmate, I will drop the **INMATEID** Column 

In [10]:
df = df.drop(columns = 'INMATEID', axis=1)
df.head()

,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SEALED,SRG_FLG,TOP_CHARGE,INFRACTION,Date,Time
0,MAX,Y,B,M,21,DE,N,Y,110-125.25,Y,2023-03-23,14:49:00
1,MED,Y,O,M,70,DE,N,N,130.50,N,2023-03-16,21:14:00
2,MED,Y,O,M,31,DE,N,Y,220.16,N,2023-06-11,00:38:00
3,NaN,Y,O,M,23,DE,N,N,110-125.25,Y,2023-03-02,01:57:34
4,MED,N,O,M,32,DE,N,N,110-125.25,N,2021-09-05,00:50:57


Here's the SQL statement alternative

In [ ]:
ALTER TABLE mytable
    DROP COLUMN DISCHARGED_DT;

#### Let's examine the values in the **CUSTODY_LEVEL** Column 

In [13]:
query = "SELECT CUSTODY_LEVEL, COUNT(*) as count FROM mytable GROUP BY CUSTODY_LEVEL;"
result = pd.read_sql_query(query, conn)

print(result)

  CUSTODY_LEVEL  count
0          None     33
1           MAX   1773
2           MED   2614
3           MIN   1726


The Medium level of security has the highest prevalence in the dataset. Minimum and Maximum security levels have roughly the same number of inmates.

#### Let's examine the values in the **BRADH** Column

Reminder: this column indicates whether or not an inmate is under mental health observations

In [14]:
query = "SELECT BRADH, COUNT(*) as count FROM mytable GROUP BY BRADH;"
result = pd.read_sql_query(query, conn)

print(result)

  BRADH  count
0     N   2913
1     Y   3233


A slim majority of the inmates are under a mental health observation. According to [MHA](https://mhanational.org/issues/state-mental-health-america) (Mental Healh America), about 21% of American adults suffer from a mental health illness. This dataset deviates from the national average by a large amount. 

#### Let's examine the values in the **Race** Column

In [15]:
query = "SELECT RACE, COUNT(*) as Count FROM mytable GROUP BY RACE;"
result = pd.read_sql_query(query, conn)

print(result)

   RACE  Count
0  None     18
1     A    146
2     B   3678
3     I     19
4     O   1763
5     U      9
6     W    513


Here's a breakdown of what race each identifying letter corresponds to:

A: Asian 

B: Black

I: American Indian 

O: Other

U: Unknown
 
W: White

Black inmates make up the vast majority of the inmate population compared to any other individual race. However, the *Other* race category still has a significant amount of inmates. It would be interesting to see the *Other* value broken down and made more specific. 

#### Let's examine the values in the **Gender** Column

In [16]:
query = "SELECT GENDER, COUNT(*) as count FROM mytable GROUP BY GENDER;"
result = pd.read_sql_query(query, conn)

print(result)

  GENDER  count
0   None     18
1      F    382
2      M   5746


A massive majority of the dataset is comprised of male inmates. Specifically, only **6%** of the dataset are females.

#### Let's examine the top 15 values in the **Age** Column

In [17]:
query = """SELECT AGE, COUNT(*) as count FROM mytable 
            GROUP BY AGE
            ORDER BY COUNT(*) DESC;"""
result = pd.read_sql_query(query, conn)

print(result.head(15))

    AGE  count
0    31    250
1    28    228
2    33    225
3    35    219
4    32    215
5    25    215
6    30    211
7    29    211
8    26    200
9    23    195
10   20    195
11   27    193
12   34    188
13   21    184
14   24    183


The top 15 age counts are within the 20-35 age range.

Let's examine some descriptive statistics 

In [18]:
query = "SELECT MIN(AGE) as Minimum_Age, MAX(AGE) as Maximum_Age, AVG(AGE) as Average_Age FROM mytable;"
result = pd.read_sql_query(query, conn)

print(result)

   Minimum_Age  Maximum_Age  Average_Age
0           18           86     35.73137


The maximum age of 86 is very high and could be the basis of further analysis. The average age of the dataset is below the national average of 39% according to the [Bureau of Justice Statistics](https://bjs.ojp.gov/content/pub/pdf/ppi16.pdf). One thing to note is that I was only able to find this information for the year 2016, so the average age may have changed significantly since then.

#### Let's examine the values in the **Inmate_Status_Code** Column

In [21]:
query = """SELECT INMATE_STATUS_CODE, COUNT(*) as Count FROM mytable 
            GROUP BY INMATE_STATUS_CODE;"""
result = pd.read_sql_query(query, conn)

print(result)

  INMATE_STATUS_CODE  Count
0                 CS    422
1                CSP      7
2                 DE   5374
3                DEP    219
4                DNS     41
5                DPV      9
6                SCO      3
7                SSR     71


Here's a breakdown of each status code and it's meaning:

- CS: City Sentenced
- CSP: City Sentenced with VP Warrant
- DE: Detainee
- DEP: Detainee - with Open Case & VP Warrant
- DNS: Detainee - Newly Sentenced to State Time
- DPV: Detainee - Technical Parole Violator
- SCO: State Prisoner - Court Order
- SSR: State Ready

A large majority of our inmates are detainees. This could indicate that most of the inmates in this dataset are newly incarcerated. 

#### Let's examine the values in the **Sealed** Column

In [22]:
query = """SELECT SEALED, COUNT(*) as Count FROM mytable 
            GROUP BY SEALED;"""
result = pd.read_sql_query(query, conn)

print(result)

  SEALED  Count
0      N   6146


Since every value is the same, I am going to drop the *SEALED* Column

In [11]:
df = df.drop(columns='SEALED', axis=1)
df.head()

,CUSTODY_LEVEL,BRADH,RACE,GENDER,AGE,INMATE_STATUS_CODE,SRG_FLG,TOP_CHARGE,INFRACTION,Date,Time
0,MAX,Y,B,M,21,DE,Y,110-125.25,Y,2023-03-23,14:49:00
1,MED,Y,O,M,70,DE,N,130.50,N,2023-03-16,21:14:00
2,MED,Y,O,M,31,DE,Y,220.16,N,2023-06-11,00:38:00
3,NaN,Y,O,M,23,DE,N,110-125.25,Y,2023-03-02,01:57:34
4,MED,N,O,M,32,DE,N,110-125.25,N,2021-09-05,00:50:57


Here's how the above code would be done in SQL

In [ ]:
ALTER TABLE mytable
    DROP COLUMN SEALED;

#### Let's examine the values in the **SRG_FLG** Column

In [25]:
query = """SELECT SRG_FLG, COUNT(*) as Count FROM mytable 
            GROUP BY SRG_FLG;"""
result = pd.read_sql_query(query, conn)

print(result)

  SRG_FLG  Count
0       N   4886
1       Y   1260


As a reminder, this column signifies if the inmate has an approved gang affiliation. The majority of the inmates do not have a gang affiliation, but there is still a significant number of inmates in a gang.

#### Let's examine the values in the **Top_Charge** Column

In [31]:
query = """SELECT TOP_CHARGE, COUNT(*) as Count FROM mytable 
            GROUP BY TOP_CHARGE
            HAVING COUNT(*) > 50
            ORDER BY COUNT(*) DESC;"""
result = pd.read_sql_query(query, conn)

print(result)

    TOP_CHARGE  Count
0       125.25    890
1   110-125.25    588
2       160.15    461
3       265.03    423
4       120.05    357
5       140.25    245
6       140.20    204
7       160.10    188
8         None    171
9       120.10    148
10      220.16    147
11      155.25     99
12      140.30     87
13  110-120.10     87
14      120.00     84
15      220.21     82
16      155.30     74
17      215.51     71
18      160.05     67
19  110-265.03     62
20      130.35     60
21      105.15     54


These values are not the most interpretable without outside sources. Upon some research, the top 2 charges are related to murder and attempted murder. I'd recommend visiting [NYSenate.gov](https://www.nysenate.gov/legislation/laws/PEN) to find any specific penal code that you are interested in further evaluating. I will be using this website any time I need to find specific descriptions of penal codes. 

#### Let's examine the values in the **Infraction** Column

In [33]:
query = """SELECT INFRACTION, COUNT(*) as Count FROM mytable 
            GROUP BY INFRACTION;"""
result = pd.read_sql_query(query, conn)

print(result)

  INFRACTION  Count
0          N   4217
1          Y   1929


The infraction column signifies if any inmate had any disciplinary actions taken against them due to their actions while imprisoned. Nearly 2000 inmates had violated prison rules, and as a result earned an infraction. 

#### Let's examine the values in the **Date** Column

If the codes below to query the date counts/values do not run, run the code directly below this cell again and try again

In [13]:
# conn = sqlite3.connect('crime_database.db')
# df.to_sql('mytable', conn, if_exists='replace', index=False)


6146

In [14]:
query = """SELECT Date, COUNT(*) as Count FROM mytable 
            GROUP BY Date
            HAVING COUNT(*) > 1
            ORDER BY COUNT(*) DESC;"""
result = pd.read_sql_query(query, conn)

print(result.head(20))

          Date  Count
0   2023-07-19     60
1   2023-07-07     58
2   2023-07-18     57
3   2023-07-13     55
4   2023-07-17     51
5   2023-06-28     50
6   2023-07-14     48
7   2023-07-03     47
8   2023-07-16     46
9   2023-07-12     46
10  2023-06-15     46
11  2023-06-29     43
12  2023-07-06     42
13  2023-06-21     42
14  2023-06-30     38
15  2023-05-04     36
16  2023-07-10     35
17  2023-07-09     35
18  2023-06-08     33
19  2023-06-06     32


Among the top 20 dates, all except one are in either June or July. Now, I'll check what the date range for the dataset is. 

In [56]:
query = """SELECT MIN(Date), MAX(Date) FROM mytable 
            ;"""
result = pd.read_sql_query(query, conn)

print(result)

    MIN(Date)   MAX(Date)
0  2012-10-28  2023-07-20


Now we know that our dates range from October 28, 2012 to July 20, 2023. It's very interesting then why all the highest arrest count dates are the most recent. I even expanded the query to list the top 50 dates and they were all still in 2023. 

#### Let's examine the values in the **Time** Column

In [64]:
query = """SELECT Time, COUNT(*) as Count FROM mytable 
            GROUP BY Time
            HAVING COUNT(*) > 1
            ORDER BY COUNT(*) DESC;"""
result = pd.read_sql_query(query, conn)

print(result.head(20))

               Time  Count
0   00:52:00.000000      8
1   01:23:00.000000      7
2   00:57:00.000000      7
3   23:57:00.000000      6
4   01:19:00.000000      6
5   21:03:00.000000      5
6   20:58:00.000000      5
7   20:47:00.000000      5
8   20:42:00.000000      5
9   18:50:00.000000      5
10  17:09:00.000000      5
11  16:56:00.000000      5
12  16:10:00.000000      5
13  01:06:00.000000      5
14  00:58:00.000000      5
15  00:50:00.000000      5
16  00:49:00.000000      5
17  00:21:00.000000      5
18  23:58:00.000000      4
19  23:35:00.000000      4


Above, I queried the times that had the highest number of arrests. Most of the times are close to midnight. Some are slightly before midnight, and some times are slightly after midnight. 

Write the DF to a csv for further analysis

In [13]:
df.to_csv('criminals.csv', index=False)

In the next file, I will extract valuable insights on the dataset through writing SQL queries. 